In [1]:
import numpy as np
import os
from sklearn.cross_validation import train_test_split
from chainer import cuda, Variable, optimizers
from PIL import Image
import glob

from model import FaceSwapNet
from model import VGG19

ModuleNotFoundError: No module named 'cPickle'

In [ ]:
def conv_setup(ORIGINAL_VGG,VGG):
    VGG.conv1_1 = ORIGINAL_VGG.conv1_1
    VGG.conv1_2 = ORIGINAL_VGG.conv1_2
    VGG.conv2_1 = ORIGINAL_VGG.conv2_1
    VGG.conv2_2 = ORIGINAL_VGG.conv2_2
    VGG.conv3_1 = ORIGINAL_VGG.conv3_1
    VGG.conv3_2 = ORIGINAL_VGG.conv3_2
    VGG.conv3_3 = ORIGINAL_VGG.conv3_3
    VGG.conv4_1 = ORIGINAL_VGG.conv4_1
    VGG.conv4_2 = ORIGINAL_VGG.conv4_2
    """
    VGG.conv4_3 = ORIGINAL_VGG.conv4_3
    VGG.conv5_1 = ORIGINAL_VGG.conv5_1
    VGG.conv5_2 = ORIGINAL_VGG.conv5_2
    VGG.conv5_3 = ORIGINAL_VGG.conv5_3
    VGG.fc6=ORIGINAL_VGG.fc6
    VGG.fc7=ORIGINAL_VGG.fc7
    """
    return VGG
    
def open_and_resize_image(content_path, style_path, target_width):
    X=[]
    for size in [8,16,32,64,128]:
        X_tmp=[]
        for path in glob.glob(content_path+"*.jpg"):
            image = Image.open(path).convert('RGB')
            X_tmp.append(np.array(image.resize((size, size), Image.BILINEAR)).transpose(2,0,1))
        X.append(X_tmp)
        
    style=[]
    for path in glob.glob(style_path+"*"):
        image = Image.open(path).convert('RGB')
        width, height = image.size
        target_height = int(round(float(height * target_width) / width))
        style.append(np.array(image.resize((target_width, target_height), Image.ANTIALIAS)).transpose(2,0,1))
    
    X=np.array(X)
    style=np.array(style)
    
    return X,style



def total_variation(x):
    xp = cuda.get_array_module(x.data)
    b, ch, h, w = x.data.shape
    wh = Variable(xp.asarray([[[[1], [-1]], [[0], [0]], [[0], [0]]], [[[0], [0]], [[1], [-1]], [[0], [0]]], [[[0], [0]], [[0], [0]], [[1], [-1]]]], dtype=xp.float32), volatile=x.volatile)
    ww = Variable(xp.asarray([[[[1, -1]], [[0, 0]], [[0, 0]]], [[[0, 0]], [[1, -1]], [[0, 0]]], [[[0, 0]], [[0, 0]], [[1, -1]]]], dtype=xp.float32), volatile=x.volatile)
    return F.sum(F.convolution_2d(x, W=wh) ** 2) + F.sum(F.convolution_2d(x, W=ww) ** 2)




vgg=VGG19()
with open("vgg19.pkl","rb") as i:
    orig_vgg19=pickle.load(i)
vgg=conv_setup(orig_vgg19,vgg)

cnn=FaceSwapNet(content_path="data/content/",style_path="data/style/",target_width=128)

X,style=load_data()
X_train=[]
X_test=[]
perm=np.random.permutation(N)
for i in range(len(X)):
    X_train.append(X[i][perm[:len(X[i])//10*9]])
    X_test.append(X[i][perm[len(X[i])//10*9:]])

optimizer=optimizer.Adam(alpha=0.001)
optimizer.setup(cnn)

cnn.to_gpu()
vgg.to_gpu()

xp=cnn.xp

N=len(X_train)
batch_size=16
kernel=3
alpha=1.0
beta=0
gamma=0.3
n_epoch=10000

style=Variable(xp.array(style,dtype=xp.float32),volatile=True)
style-=xp.array([[[[124]],[[117]],[[104]]]])
style_feature=vgg(style)
style_patch=[]

for name in ["3_1","4_1"]:
    patch=xp.array([style_feature[name][0,:,i:i+kernel,j:j+kernel].data/xp.lilalg.norm(style_feature[name][0,:,i:i+kernel,j:j+kernel].data) for i in range(style_feature[name].shape[2]-kernel+1) for j in range(style_feature[name].shape[3]-kernel+1)])
    style_patch.append(patch)
del patch


for epoch in range(1,n_epoch+1):
    print("epoch",epoch)
    perm=xp.random.permutation(N)
    L=0
    if epoch in np.arange(1,21)*20:
        beta+=1
    for i in range(0,N,batch_size):
        x1=xp.array(X_train[0]perm[i,i:i+batch_size]/127.5-1.).astype(xp.float32)
        x2=xp.array(X_train[1]perm[i,i:i+batch_size]/127.5-1.).astype(xp.float32)
        x3=xp.array(X_train[2]perm[i,i:i+batch_size]/127.5-1.).astype(xp.float32)
        x4=xp.array(X_train[3]perm[i,i:i+batch_size]/127.5-1.).astype(xp.float32)
        x5=xp.array(X_train[4]perm[i,i:i+batch_size]/127.5-1.).astype(xp.float32)
        
        optimizer.zero_grads()
        
        swap_X=cnn(x1,x2,x3,x4,x5)
        contents=Variable(xp.array(X_train[-1][perm[i,i:i+batch_size]]),volatile=True)
        swap_X-=xp.array([[[[124]],[[117]],[[104]]]])
        contents-=xp.array([[[[124]],[[117]],[[104]]]])
        
        swap_feature=vgg(swap_X)
        content_feature=vgg(contents)
        
        L_content=F.mean_squared_error(Variable(content_feature["4_2"].data), swap_feature["4_2"])
        L_style=0
        for s,name in enumerate(["3_1","4_1"]):
            L_style+=cnn.local_patch(swap_X[name],xp.arrray(style_patch[s],dtype=xp.float32))
        L_style/=s
        L_tv=total_variation(swap_X)
        L=alpha*L_content+beta*L_style+gamma*L_tv
        
        L.backward()
        optimizer.update()
    
    print("train mean loss={}".format(L/N))
        